<a href="https://colab.research.google.com/github/ahsaniqbal/CharacterLevelLM/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

In [0]:
print(torch.__version__)

1.1.0


In [0]:
tensor = torch.tensor([[[11, 11], [21, 21], [31, 31]], [[12, 12], [22, 22], [32, 32]]])

In [0]:
tensor.shape

torch.Size([2, 3, 2])

In [0]:
b = torch.tensor([[1,0], [0, 1]])

In [0]:
torch.mm(tensor.view(-1, 2), b).view(2, 3, 2)

tensor([[[11, 11],
         [21, 21],
         [31, 31]],

        [[12, 12],
         [22, 22],
         [32, 32]]])